<a href="https://colab.research.google.com/github/millermuttu/Notebooks/blob/master/traffic_volume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor as xgbr # for modelling
from sklearn.pipeline import Pipeline # for making pipleine 
from sklearn.impute import SimpleImputer # for handling missing variables either categorical or numerical
from sklearn.preprocessing import OneHotEncoder # for one hot encoding categorical variables
from sklearn.metrics import mean_absolute_error # for Mean absolute error
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
import os
from sklearn.preprocessing import OneHotEncoder

In [93]:
os.curdir

'.'

In [94]:
train_data = pd.read_csv('Train.csv')
train_data.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [0]:
date = pd.to_datetime(train_data['date_time'])
date_encoded = []
for i in date:
    day = i.strftime('%A')
    time = i.hour
    day_time = str(day)+str(time)
    # date_encoded.append([day+time])
    date_encoded.append([day_time])

date_encoded = pd.DataFrame(date_encoded)
train_data['date_time'] = date_encoded

train_data['date_time'] = train_data['date_time'].astype('category').cat.codes
# train_data['is_holiday'] = train_data['is_holiday'].astype('category').cat.codes
# train_data['weather_type'] = train_data['weather_type'].astype('category').cat.codes
# train_data['weather_description'] = train_data['weather_description'].astype('category').cat.codes

In [79]:
train_data.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,143,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,122,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,123,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,124,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,125,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [96]:
train_data.dtypes

date_time                int16
is_holiday              object
air_pollution_index      int64
humidity                 int64
wind_speed               int64
wind_direction           int64
visibility_in_miles      int64
dew_point                int64
temperature            float64
rain_p_h               float64
snow_p_h               float64
clouds_all               int64
weather_type            object
weather_description     object
traffic_volume           int64
dtype: object

In [0]:
test_data = pd.read_csv('Test.csv')

In [81]:
test_data.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,2017-05-18 00:00:00,None,73,63,1,27,4,4,285.15,0.0,0,90,Rain,moderate rain
1,2017-05-18 00:00:00,None,251,63,1,27,4,4,285.15,0.0,0,90,Mist,mist
2,2017-05-18 00:00:00,None,75,56,1,0,1,1,285.15,0.0,0,90,Drizzle,light intensity drizzle
3,2017-05-18 01:00:00,None,98,56,1,351,2,2,284.79,0.0,0,90,Rain,heavy intensity rain
4,2017-05-18 01:00:00,None,283,56,1,351,1,1,284.79,0.0,0,90,Mist,mist


In [0]:
date = pd.to_datetime(test_data['date_time'])
date_encoded = []
for i in date:
    day = i.strftime('%A')
    time = i.hour
    day_time = str(day)+str(time)
    # date_encoded.append([day+time])
    date_encoded.append([day_time])

date_encoded = pd.DataFrame(date_encoded)
test_data['date_time'] = date_encoded

test_data['date_time'] = test_data['date_time'].astype('category').cat.codes

In [99]:
test_data.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,96,None,73,63,1,27,4,4,285.15,0.0,0,90,Rain,moderate rain
1,96,None,251,63,1,27,4,4,285.15,0.0,0,90,Mist,mist
2,96,None,75,56,1,0,1,1,285.15,0.0,0,90,Drizzle,light intensity drizzle
3,97,None,98,56,1,351,2,2,284.79,0.0,0,90,Rain,heavy intensity rain
4,97,None,283,56,1,351,1,1,284.79,0.0,0,90,Mist,mist


In [100]:
train_data.columns
train_data.shape

(33750, 15)

In [0]:
y = train_data.traffic_volume                 # seperating the regression value to be predicted

In [0]:
train_data.drop(['traffic_volume'], axis=1, inplace=True)

In [0]:
x_train,x_test,y_train,y_test = train_test_split(train_data,y,train_size=0.85,random_state= 0)

In [104]:
print("shape of x_train",x_train.shape)
print("shape of y_train",y_train.shape)
print("shape of x_test",x_test.shape)
print("shape of y_test",y_test.shape)

shape of x_train (28687, 14)
shape of y_train (28687,)
shape of x_test (5063, 14)
shape of y_test (5063,)


In [0]:
categorical_cols = [cname for cname in x_train.columns if 
                   x_train[cname].nunique()<40 and
                   x_train[cname].dtype == 'object']

In [0]:
numerical_cols = [cname for cname in x_train.columns if
                 x_train[cname].dtype in ['int64', 'float64']]

In [107]:
print(categorical_cols,numerical_cols)

['is_holiday', 'weather_type', 'weather_description'] ['air_pollution_index', 'humidity', 'wind_speed', 'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature', 'rain_p_h', 'snow_p_h', 'clouds_all']


In [108]:
x_train.weather_description.nunique()

38

In [88]:
x_train.is_holiday.dtype

dtype('int8')

In [0]:
my_cols = categorical_cols + numerical_cols
x_train_1 = x_train[my_cols].copy()
x_test_1 = x_test[my_cols].copy()

In [0]:


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# handle unknown is set to ignore because sometimes in test set we have variables that were not present in Training set and hence were not encoded while training 
# but if we use these variables while testing we will get error hence to ignore these errors we use this argument

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



In [0]:
model = xgbr(random_state=42, learning_rate=0.055,n_estimators=2000)

In [0]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [127]:
clf.fit(x_train_1, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:10:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  ['air_pollution_index',
                                                   'humidity', 'wind_speed',
                                                   'wind_direction',


In [128]:
# Preprocessing of validation data, get predictions
preds = clf.predict(x_test_1)

print('MAE:', mean_absolute_error(y_test, preds))

MAE: 1604.242410740032


In [129]:
y_test,preds

(10802    2789
 4407     4814
 7745     3238
 12064     375
 22623    1381
          ... 
 15478    4182
 6804     4058
 4889     4766
 13945    3235
 9758     4319
 Name: traffic_volume, Length: 5063, dtype: int64,
 array([2666.3596, 2731.0757, 4012.5195, ..., 3597.0083, 3432.2964,
        3227.4722], dtype=float32))

In [130]:
len(preds)

5063